In [ ]:
import pandas as pd
import pyam
import nomenclature

In [ ]:
iamc_args = dict(
    model="State of CDR (2024) - Voluntary Carbon Market",
    scenario="Historical Reference",
)

In [ ]:
dsd = nomenclature.DataStructureDefinition("../definitions/")

In [ ]:
df_list = list()

## Indicator 17

In [ ]:
for sheet_name, _type in (("17(a)- Conventional Issuances", "Issued"), ("17(b)-Conventional Retirements", "Sold")):
    data = pd.read_excel(
        "source/4_Voluntary_Carbon_Market/C4- Updated Full Dashboard & Summary Data.xlsx",
        sheet_name=sheet_name,
        header=0,
        nrows=5,
        usecols="A:C",
    )
    data.columns = ["variable", 2022, 2023]
    
    df_method = pyam.IamDataFrame(
        data,
        **iamc_args,
        unit="t CO2",
        region="World",
    )
    df_method.rename(
        variable={
            "Afforestation/Reforestation": "Afforestation and Reforestation",
            "Wetland Restoration": "Peatland and Wetland Restoration",
            "Wooden Building Elements": "Durable Wood Products",
        },
        inplace=True,
    )
    df_method.rename(
        variable=dict([(i, f"Carbon Credits|Conventional CDR|Tons {_type}|{i}") for i in df_method.variable]),
        inplace=True,
    )
    df_method.aggregate(f"Carbon Credits|Conventional CDR|Tons {_type}", append=True)

    data = pd.read_excel(
        "source/4_Voluntary_Carbon_Market/C4- Updated Full Dashboard & Summary Data.xlsx",
        sheet_name=sheet_name,
        header=9,
        nrows=37,
        usecols="A:C",
    )
    data.columns = ["region", 2022, 2023]
    
    df_country = pyam.IamDataFrame(
        data, 
        **iamc_args,
        variable=f"Carbon Credits|Conventional CDR|Tons {_type}",
        unit="t CO2",
    )

    df_list.append(pyam.concat([df_method, df_country]))

## Indicator 18

In [ ]:
for sheet_name, _type in (("18(a)-Novel Purchases", "Sold"), ("18(b)- Novel Deliveries", "Delivered")):
    data = pd.read_excel(
        "source/4_Voluntary_Carbon_Market/C4- Updated Full Dashboard & Summary Data.xlsx",
        sheet_name=sheet_name,
        header=0,
        nrows=11,
        usecols="A:C",
    )
    data.columns = ["variable", 2022, 2023]
    
    df_method = pyam.IamDataFrame(
        data,
        **iamc_args,
        unit="t CO2",
        region="World",
    )
    df_method.rename(
        variable={
            "BECCS": "Bioenergy with CCS",
            "Biomass Removal": "Biomass Burial",
            "Biooil": "Bio-Oil Storage",
            "DAC": "Direct Air Capture with CCS",
            "Direct Ocean Removal": "Direct Ocean Capture",
            "Microalgae": "Ocean Fertilization",
            "Not disclosed": "Not Disclosed",
        },
        inplace=True,
    )
    df_method.rename(
        variable=dict([(i, f"Carbon Credits|Novel CDR|Tons {_type}|{i}") for i in df_method.variable]),
        inplace=True,
    )
    df_method.aggregate(f"Carbon Credits|Novel CDR|Tons {_type}", append=True)

    df_list.append(pyam.concat([df_method]))

## Merge data

In [ ]:
df = pyam.concat(df_list)

In [ ]:
df.rename(region={"Republic of Congo": "Congo"}, inplace=True)

In [ ]:
df.convert_unit("t CO2", "Mt CO2", inplace=True)

In [ ]:
dsd.validate(df)

In [ ]:
df.filter(region="World", variable="Carbon Credits|Conventional CDR|Tons Sold|*").plot.bar(stacked=True)

In [ ]:
df.variable

# Import to ixmp4 database

In [ ]:
import ixmp4

In [ ]:
platform = ixmp4.Platform("socdr-dev")

In [ ]:
run = platform.runs.get(**iamc_args)

In [ ]:
run.iamc.remove(run.iamc.tabulate())

In [ ]:
run.iamc.add(df.data)

## Figure 4.3

In [ ]:
data = pd.read_excel(
    "source/4_Voluntary_Carbon_Market/C4 Final Data.xlsx",
    sheet_name="data_both_4.3",
    header=1,
    nrows=12,
    usecols="A:D",
)

In [ ]:
for i in range(6):
    data.loc[2*i+1, "SoCDR Classification"] = data.loc[2*i, "SoCDR Classification"]

In [ ]:
_df = pyam.IamDataFrame(
    data, 
    model="State of CDR (2024) - Voluntary Carbon Market",
    scenario="Historical Reference",
    unit="t CO2",
    region="World",
    value=["Tons Sold", "Tons Issued"],
)
_data = _df.data.rename(columns={"variable": "category"})

df = pyam.IamDataFrame(
    _data,
    variable=["category", "socdr classification"]
).convert_unit("t CO2", "Mt CO2")

df.rename(variable=dict({
    (i, i.replace("Tons Issued|", "Carbon Credits|Issued Credits|").replace("Tons Sold|", "Carbon Credits|Sold Credits|"))
    for i in df.variable
}), inplace=True)

df.aggregate(["Carbon Credits|Issued Credits", "Carbon Credits|Sold Credits"], append=True)

In [ ]:
df_list.append(df)

## Figure 4.4a

In [ ]:
data = pd.read_excel(
    "source/4_Voluntary_Carbon_Market/C4 Final Data.xlsx",
    sheet_name="data_both_4.4.1",
    header=2,
    nrows=4,
    usecols="A:H",
)

In [ ]:
for i in range(2):
    data.loc[2*i+1, "Year"] = data.loc[2*i, "Year"]

In [ ]:
_df = pyam.IamDataFrame(
    data, 
    model="State of CDR (2024) - Voluntary Carbon Market",
    scenario="Historical Reference",
    unit="t CO2",
    region="World",
    value=["Afforestation/Reforestation", "Carbonated Building Elements", "Improved Forest Management",
       "Soil Carbon Sequestration", "Wetland Restoration", "Wooden Building Elements"],
)
_data = _df.data.rename(columns={"variable": "category"})

df = pyam.IamDataFrame(
    _data,
    variable=["sum", "category"]
).convert_unit("t CO2", "Mt CO2")

In [ ]:
df.rename(variable=dict({
    (
        i, 
        i.replace("Tons Issued|", "Carbon Credits|Issued Credits For All Conventional CDR|")
        .replace("Tons Sold|", "Carbon Credits|Sold Credits For All Conventional CDR|")
    )
    for i in df.variable
}), inplace=True)

In [ ]:
df_list.append(df)

In [ ]:
df.aggregate(
    ["Carbon Credits|Sold Credits For All Conventional CDR", "Carbon Credits|Issued Credits For All Conventional CDR"]
).timeseries()

In [ ]:
pyam.concat(df_list).filter(variable=["Carbon Credits|Sold Credits|Conventional CDR", "Carbon Credits|Issued Credits|Conventional CDR"]).timeseries()

## Figure 4.4b

In [ ]:
data = pd.read_excel(
    "source/4_Voluntary_Carbon_Market/C4 Final Data.xlsx",
    sheet_name="data_both_4.4.2",
    header=3,
    nrows=4,
    usecols="A:M",
)

In [ ]:
for i in range(2):
    data.loc[2*i+1, "Year"] = data.loc[2*i, "Year"]

In [ ]:
data.columns

In [ ]:
_df = pyam.IamDataFrame(
    data, 
    model="State of CDR (2024) - Voluntary Carbon Market",
    scenario="Historical Reference",
    unit="t CO2",
    region="World",
    value=["BECCS", "Biochar", "Biomass Removal", "Biooil", "DAC",
       "Direct Ocean Removal", "Enhanced Weathering", "Fertilsation",
       "Mineralisation", "Not disclosed", "Ocean Alkalinity Enhancement"],
)
_data = _df.data.rename(columns={"variable": "category"})

df = pyam.IamDataFrame(
    _data,
    variable=["sum", "category"]
).convert_unit("t CO2", "Mt CO2")

In [ ]:
df.rename(variable=dict({
    (
        i, 
        i.replace("Tons Delivered|", "Carbon Credits|Issued Credits For All Novel CDR|")
        .replace("Tons Sold|", "Carbon Credits|Sold Credits For All Novel CDR|")
    )
    for i in df.variable
}), inplace=True)

In [ ]:
df_list.append(df)

In [ ]:
final_df = pyam.concat(df_list)

In [ ]:
final_df.rename(variable=dict({
    (
        i, 
        i.replace("Afforestation/Reforestation", "Afforestation and Reforestation")
        .replace("Improved Forest Management", "Forest Management")
        .replace("Wetland Restoration", "Peatland and Wetland Restoration")
        .replace("Biomass Removal", "Biomass Burial")
        .replace("DAC", "Direct Air Capture")
        .replace("Fertilsation", "Ocean Fertilization")
        .replace("Direct Ocean Removal", "Direct Ocean Capture")
        .replace("Not disclosed", "Not Disclosed")
#        .replace("Mineralisation", "")       
    )
    for i in final_df.variable
}), inplace=True)

In [ ]:
dsd = nomenclature.DataStructureDefinition("../definitions/")

In [ ]:
dsd.validate(final_df)

In [ ]:
final_df.to_excel("SoCDR-Chapter-4-data.xlsx")

In [ ]:
df.set_meta("Chapter 4", "Chapter")

In [ ]:
#df.to_ixmp4("socdr-dev")

In [ ]:
df.unit

In [ ]:
import ixmp4

In [ ]:
platform = ixmp4.Platform("socdr-dev")

In [ ]:
platform.units.tabulate()

In [ ]:
platform.units.create("Mt CO2")

In [ ]:
pyam.__version__